In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ["LANGSMITH_TRACING"] = "true"

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='o1-mini')
llm

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
file_path = 'An overview of gradient descent optimazation algorithms.pdf'
loader = PyPDFLoader(file_path)
docs = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 5000, chunk_overlap = 600)

In [ ]:
document = splitter.split_documents(docs)

In [ ]:
document

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

vector_store = FAISS.from_documents(document, embeddings)



In [ ]:
retriever = vector_store.as_retriever()

In [52]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
  input_variables=['context', 'question'],
  template="""
  Answer the following question based on the given context:
  <context>
  {context}
  </context>

  Question: {question}
  Answer:
  """
)


In [53]:
from langchain.chains.combine_documents import create_stuff_documents_chain

chain = create_stuff_documents_chain(
  llm = llm,
  prompt = prompt,
)

In [54]:
print(type(retriever))  # Check the actual type
print(retriever)        # See the contents


<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>
tags=['FAISS', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023ACCBF4090> search_kwargs={}


In [56]:
question = "What is this document about?"
context_docs = retriever.get_relevant_documents(question)  # Retrieve actual documents


response = chain.invoke(
{
  "context": context_docs,
  "question": question
}
)

In [57]:
print(response)

The document provides a comprehensive overview of gradient descent optimization algorithms, primarily focusing on their application in training neural networks. It explores various gradient descent variants, such as mini-batch gradient descent, and delves into the challenges encountered during the training process. The article introduces and explains common optimization algorithms like Momentum, Nesterov Accelerated Gradient, Adagrad, Adadelta, RMSprop, Adam, AdaMax, and Nadam, highlighting their motivations and update rules. Additionally, it examines strategies for optimizing gradient descent in parallel and distributed settings and discusses supplementary techniques such as shuffling, curriculum learning, batch normalization, early stopping, and the addition of gradient noise to enhance the performance and robustness of these optimization methods.
